In [360]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import yfinance as yf
import seaborn as sns

In [361]:
#Importing dataset and defining path variables
path = './data/'
data_2018 = 'ipo_stock_2010_2018.csv'
data_2019 = 'ipo_stock_2019.csv'
volume_data = 'ipo_stock_2019_volumes.csv'


#Reading in file 1 data
df_ipo = pd.read_csv(path + data_2018)

#Reading in file 2 data
cols = ['Symbol', 'ipoDate', 'openDay0', 'closeDay0',
 'closeDay6', 'closeDay29', 'dayOfWeek', 'employees',
  'Name', 'CEOAge', 'Sector']
df_ipo_2 = pd.read_csv(path + data_2019, encoding= 'ISO-8859-1', usecols= cols)

df_ipo

,Company Name,Symbol,Market,Price,Shares,Offer Amount,Date Priced,employees,address,US_state,...,inmonth_volume,inweek_adjclose,inweek_open,inweek_spread,inweek_volume,sector,industry,employees2019,CEO_pay,CEO_born
0,ANDATEE CHINA MARINE FUEL SERVICES CORP,AMCF,NASDAQ,6.3,3134921.0,1.975000e+07,2010-01-26,128.0,NO. 68 BINHAI RD DALIAN XIGANG DISTRICTDALIAN ...,NaN,...,323300.0,5.830000,6.030000,0.280000,106300.0,Energy,Oil & Gas Equipment & Services,189.0,NaN,NaN
1,CHESAPEAKE LODGING TRUST,CHSP,New York Stock Exchange,20.0,7500000.0,1.500000e+08,2010-01-22,3.0,"4300 WILSON BOULEVARDSUITE 625ARLINGTON, VA 22203",VA,...,40500.0,11.927483,18.910000,0.139999,75900.0,Real Estate,REIT - Hotel & Motel,13.0,2320000.0,1962.0
2,GENERAC HOLDINGS INC.,GNRC,New York Stock Exchange,13.0,18750000.0,2.437500e+08,2010-02-11,1486.0,"S45 W29290 HIGHWAY 59WAUKESHA, WI 53187",WI,...,265400.0,8.612183,13.050000,0.290000,110000.0,Industrials,Diversified Industrials,5046.0,1650000.0,1972.0
3,"QUINSTREET, INC",QNST,NASDAQ,15.0,10000000.0,1.500000e+08,2010-02-11,568.0,"950 TOWER LANE, 6TH FLOORFOSTER CITY, CA 94404",CA,...,27300.0,12.980000,13.500000,0.809999,504000.0,Technology,Internet Content & Information,506.0,816150.0,1960.0
4,TERRENO REALTY CORP,TRNO,New York Stock Exchange,20.0,8750000.0,1.750000e+08,2010-02-10,6.0,"16 MAIDEN LANEFIFTH FLOORSAN FRANCISCO, CA 94108",CA,...,64300.0,14.770803,18.809999,0.199999,91200.0,Real Estate,REIT - Industrial,23.0,799500.0,1961.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,TENCENT MUSIC ENTERTAINMENT GROUP,TME,NYSE,13.0,82000000.0,1.066000e+09,2018-12-12,2559.0,"17/F, MALATA BUILDING, KEJIZHONGYI ROADMIDWEST...",NaN,...,5211900.0,12.190000,12.400000,0.640000,6873900.0,Technology,Internet Content & Information,3041.0,NaN,1974.0
1132,"SYNTHORX, INC.",THOR,NASDAQ Global Select,11.0,11912727.0,1.310400e+08,2018-12-07,25.0,11099 NORTH TORREY PINES ROADSUITE 190LA JOLLA...,CA,...,86800.0,14.990000,15.300000,1.676001,84500.0,Healthcare,Biotechnology,39.0,606480.0,1958.0
1133,DIAMEDICA THERAPEUTICS INC.,DMAC,NASDAQ Capital,4.0,4100000.0,1.640000e+07,2018-12-07,11.0,"2 CARLSON PARKWAYSUITE 260MINNEAPOLIS, MN 55447",MN,...,268100.0,8.744000,7.940000,0.944000,419500.0,Healthcare,Biotechnology,9.0,492400.0,1971.0
1134,"MODERNA, INC.",MRNA,NASDAQ Global Select,23.0,26275993.0,6.043478e+08,2018-12-07,700.0,"200 TECHNOLOGY SQUARECAMBRIDGE, MA 02139",MA,...,652600.0,17.000000,17.100000,1.000000,1621600.0,Healthcare,Biotechnology,760.0,2670000.0,1973.0


In [362]:
#Imputing missing values with mean for numerical columns
numeric_cols = df_ipo.select_dtypes(include=['int64','float64'])
categorical_cols = df_ipo.select_dtypes(include=['object'])

#Imputing numeric cols
numeric_cols.fillna(numeric_cols.mean(), inplace=True)
categorical_cols = categorical_cols.apply(lambda x: x.fillna(x.mode()[0]) if x.isna().sum() > 0 else x, axis = 0)


In [363]:
#Removing CEO birth year and creating CEO age feature
numeric_cols.CEO_born = 2022 - numeric_cols.CEO_born
numeric_cols.rename(columns = {'CEO_born': 'CEO_age'}, inplace= True)

#Building binary flags for three return periods (1 for underperformance, 0 for non underperformance, e.g. positive returns)
numeric_cols['intra_day_up'] = numeric_cols.apply(lambda x: int(x['firstday_adjclose'] < x['Price']), axis = 1)
numeric_cols['intra_week_up'] = numeric_cols.apply(lambda x: int(x['inweek_adjclose'] < x['Price']), axis = 1)
numeric_cols['intra_month_up'] = numeric_cols.apply(lambda x: int(x['inmonth_adjclose'] < x['Price']), axis = 1)

#Adding day of the week feature
categorical_cols['IPO_weekday'] = pd.to_datetime(categorical_cols['Date Priced']).dt.day_of_week


In [364]:
#Performing Shapiro Wilk tests to test for approximate normality
import math
import numpy as np
from scipy.stats import shapiro 
from scipy.stats import kstest
from scipy.stats import lognorm

normality_check = numeric_cols.apply(lambda x: print(kstest(x, 'norm')), axis = 0)
#normality_check = numeric_cols.apply(lambda x: print(shapiro(x)), axis = 0)

#Most columns are not Gaussian, normalization instead of standardization is used

KstestResult(statistic=0.999088047068026, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)
KstestResult(statistic=0.9431923554894966, pvalue=0.0)
KstestResult(statistic=0.9514588029717197, pvalue=0.0)
KstestResult(statistic=0.9540996480421422, pvalue=0.0)
KstestResult(statistic=0.5, pvalue=3.867951959080918e-263)
KstestResult(statistic=0.905809859154929, pvalue=0.0)
KstestResult(statistic=0.9604730383062184, pvalue=0.0)
KstestResult(statistic=0.9616243281580873, pvalue=0.0)
KstestResult(statistic=0.5, pvalue=3.867951959080918e-263)
KstestResult(statistic=0.9595070422535211, pvalue=0.0)
KstestResult(statistic=0.9573605740545975, pvalue=0.0)
KstestResult(statistic=0.9587706671616072, pvalue=0.0)
KstestResult(statistic=0.5, pvalue=3.867951959080918e-263)
KstestResult(statistic=0.9559859154929577, pvalue=0.0)
KstestResult(statistic=0.9973274836877444, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)
KstestResult(statistic=1.0, pvalue=0.0)
Kstes

In [365]:
#Concatenating imputed cols
df_ipo = pd.concat([numeric_cols, categorical_cols], axis = 1)
df_ipo.isna().sum()

Price                0
Shares               0
Offer Amount         0
employees            0
firstday_adjclose    0
firstday_open        0
firstday_spread      0
firstday_volume      0
inmonth_adjclose     0
inmonth_open         0
inmonth_spread       0
inmonth_volume       0
inweek_adjclose      0
inweek_open          0
inweek_spread        0
inweek_volume        0
employees2019        0
CEO_pay              0
CEO_age              0
intra_day_up         0
intra_week_up        0
intra_month_up       0
Company Name         0
Symbol               0
Market               0
Date Priced          0
address              0
US_state             0
descriptions         0
link_nasdaq          0
year                 0
sector               0
industry             0
IPO_weekday          0
dtype: int64

In [366]:
#Looking at class balances
print(df_ipo.intra_day_up.value_counts(normalize=True))
print(df_ipo.intra_week_up.value_counts(normalize=True))
print(df_ipo.intra_month_up.value_counts(normalize=True))

0    0.613556
1    0.386444
Name: intra_day_up, dtype: float64
0    0.587148
1    0.412852
Name: intra_week_up, dtype: float64
0    0.576585
1    0.423415
Name: intra_month_up, dtype: float64


In [367]:
#Dropping unwanted / inaccurate / incomplete cols
drop_columns = ['Offer Amount', 'firstday_adjclose', 'firstday_volume','inmonth_volume', 'inweek_volume',
       'firstday_open', 'firstday_spread', 'inmonth_adjclose', 'inmonth_open', 'inmonth_spread',
        'inweek_adjclose', 'inweek_open', 'inweek_spread','employees2019', 'firstday_volume',
       'Company Name','address', 'US_state', 'descriptions', 'link_nasdaq','year', 'industry', 'year', 'industry']
df_ipo.drop(drop_columns, axis = 1, inplace = True)
df_ipo

,Price,Shares,employees,CEO_pay,CEO_age,intra_day_up,intra_week_up,intra_month_up,Symbol,Market,Date Priced,sector,IPO_weekday
0,6.3,3134921.0,128.0,1.232574e+06,57.460211,1,1,0,AMCF,NASDAQ,2010-01-26,Energy,1
1,20.0,7500000.0,3.0,2.320000e+06,60.000000,1,1,1,CHSP,New York Stock Exchange,2010-01-22,Real Estate,4
2,13.0,18750000.0,1486.0,1.650000e+06,50.000000,1,1,1,GNRC,New York Stock Exchange,2010-02-11,Industrials,3
3,15.0,10000000.0,568.0,8.161500e+05,62.000000,0,1,0,QNST,NASDAQ,2010-02-11,Technology,3
4,20.0,8750000.0,6.0,7.995000e+05,61.000000,1,1,1,TRNO,New York Stock Exchange,2010-02-10,Real Estate,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1131,13.0,82000000.0,2559.0,1.232574e+06,48.000000,0,1,0,TME,NYSE,2018-12-12,Technology,2
1132,11.0,11912727.0,25.0,6.064800e+05,64.000000,0,0,0,THOR,NASDAQ Global Select,2018-12-07,Healthcare,4
1133,4.0,4100000.0,11.0,4.924000e+05,51.000000,0,0,0,DMAC,NASDAQ Capital,2018-12-07,Healthcare,4
1134,23.0,26275993.0,700.0,2.670000e+06,49.000000,1,1,1,MRNA,NASDAQ Global Select,2018-12-07,Healthcare,4


In [368]:
#Creating trading volume features from trading volume dataset and yfinance data

#Reading in trading volume data (contains about 900 of the total 1100 companies)
df_ipo_volumes = pd.read_csv(path + volume_data, sep = ';')

#Removing duplicate rows
df_ipo_volumes.drop_duplicates(inplace=True)

#Computing inweek and inmonth volumes
df_ipo_volumes['inweek_volume'] = df_ipo_volumes.loc[:, 'volumeDay1':'volumeDay6'].apply(lambda x: sum(x), axis = 1)
df_ipo_volumes['inmonth_volume'] = df_ipo_volumes.loc[:, 'volumeDay1': 'volumeDay29'].apply(lambda x: sum(x), axis = 1)
df_ipo_volumes = df_ipo_volumes[['Symbol', 'volumeDay0', 'inweek_volume', 'inmonth_volume']]
df_ipo_volumes

,Symbol,volumeDay0,inweek_volume,inmonth_volume
0,A,59753154.0,33621764.0,99043170.0
1,AAC,2799073.0,706741.0,3618453.0
2,AAOI,948999.0,688968.0,2730935.0
3,AAP,371100.0,1358700.0,6440400.0
4,AAT,15536889.0,4181293.0,10410871.0
...,...,...,...,...
3757,ZUMZ,6793424.0,1954885.0,7014295.0
3758,ZUO,13102419.0,5574967.0,NaN
3759,ZX,4987284.0,8667411.0,12673532.0
3760,ZYME,953887.0,271634.0,886679.0


In [369]:
#Renaming cols for dataset merging
df_ipo_2.rename(columns = {'dayOfWeek':'IPO_weekday',
 'openDay0':'Price',
  'volumeDay0':'firstday_volume',
    'CEOAge':'CEO_age',
    'Sector': 'sector'}, inplace=True)

#Adding computed trading volumes
df_ipo_2['firstday_volume'] = df_ipo_volumes['volumeDay0']
df_ipo_2['inweek_volume'] = df_ipo_volumes['inweek_volume']
df_ipo_2['inmonth_volume'] = df_ipo_volumes['inmonth_volume']

df_ipo_2

,Symbol,IPO_weekday,closeDay0,Price,closeDay6,closeDay29,Name,sector,ipoDate,CEO_age,employees,firstday_volume,inweek_volume,inmonth_volume
0,A,3,28.6358,27.3725,27.7518,46.5332,"Agilent Technologies, Inc.",Capital Goods,11/18/99 0:00,56.0,13500,59753154.0,33621764.0,99043170.0
1,AAC,3,18.5000,17.6000,19.0000,25.0000,"AAC Holdings, Inc.",Health Care,10/2/14 0:00,46.0,2100,2799073.0,706741.0,3618453.0
2,AAOI,3,9.9600,9.3700,10.2000,13.1200,"Applied Optoelectronics, Inc.",Technology,9/26/13 0:00,54.0,3054,948999.0,688968.0,2730935.0
3,AAP,3,13.9000,13.3833,14.6670,13.7330,Advance Auto Parts Inc,Consumer Services,11/29/01 0:00,59.0,71000,371100.0,1358700.0,6440400.0
4,AAT,3,21.2500,21.1800,21.2500,21.2200,"American Assets Trust, Inc.",Consumer Services,1/13/11 0:00,79.0,194,15536889.0,4181293.0,10410871.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,ZUMZ,4,12.4350,10.2500,11.5810,15.1500,Zumiez Inc.,Consumer Services,5/6/05 0:00,57.0,8900,6793424.0,1954885.0,7014295.0
3758,ZUO,3,20.0000,19.1200,19.7200,NaN,"Zuora, Inc.",NaN,4/12/18 0:00,NaN,933,13102419.0,5574967.0,NaN
3759,ZX,3,5.8500,5.6000,6.0095,5.3500,China Zenix Auto International Limited,Capital Goods,5/12/11 0:00,47.0,-,4987284.0,8667411.0,12673532.0
3760,ZYME,4,13.0000,13.0000,13.0300,9.6200,Zymeworks Inc.,Health Care,4/28/17 0:00,NaN,147,953887.0,271634.0,886679.0


In [370]:
#Adding underperformance labels
df_ipo_2['intra_day_up'] = df_ipo_2.apply(lambda x: int(x['closeDay0'] < x['Price']), axis = 1)
df_ipo_2['intra_week_up'] = df_ipo_2.apply(lambda x: int(x['closeDay6'] < x['Price']), axis = 1)
df_ipo_2['intra_month_up'] = df_ipo_2.apply(lambda x: int(x['closeDay29'] < x['Price']), axis = 1)
df_ipo_2

,Symbol,IPO_weekday,closeDay0,Price,closeDay6,closeDay29,Name,sector,ipoDate,CEO_age,employees,firstday_volume,inweek_volume,inmonth_volume,intra_day_up,intra_week_up,intra_month_up
0,A,3,28.6358,27.3725,27.7518,46.5332,"Agilent Technologies, Inc.",Capital Goods,11/18/99 0:00,56.0,13500,59753154.0,33621764.0,99043170.0,0,0,0
1,AAC,3,18.5000,17.6000,19.0000,25.0000,"AAC Holdings, Inc.",Health Care,10/2/14 0:00,46.0,2100,2799073.0,706741.0,3618453.0,0,0,0
2,AAOI,3,9.9600,9.3700,10.2000,13.1200,"Applied Optoelectronics, Inc.",Technology,9/26/13 0:00,54.0,3054,948999.0,688968.0,2730935.0,0,0,0
3,AAP,3,13.9000,13.3833,14.6670,13.7330,Advance Auto Parts Inc,Consumer Services,11/29/01 0:00,59.0,71000,371100.0,1358700.0,6440400.0,0,0,0
4,AAT,3,21.2500,21.1800,21.2500,21.2200,"American Assets Trust, Inc.",Consumer Services,1/13/11 0:00,79.0,194,15536889.0,4181293.0,10410871.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3757,ZUMZ,4,12.4350,10.2500,11.5810,15.1500,Zumiez Inc.,Consumer Services,5/6/05 0:00,57.0,8900,6793424.0,1954885.0,7014295.0,0,0,0
3758,ZUO,3,20.0000,19.1200,19.7200,NaN,"Zuora, Inc.",NaN,4/12/18 0:00,NaN,933,13102419.0,5574967.0,NaN,0,0,0
3759,ZX,3,5.8500,5.6000,6.0095,5.3500,China Zenix Auto International Limited,Capital Goods,5/12/11 0:00,47.0,-,4987284.0,8667411.0,12673532.0,0,0,1
3760,ZYME,4,13.0000,13.0000,13.0300,9.6200,Zymeworks Inc.,Health Care,4/28/17 0:00,NaN,147,953887.0,271634.0,886679.0,0,0,1


In [371]:
#Converting dtypes
df_ipo_2.employees = df_ipo_2.employees.replace('-', 0)
df_ipo_2.employees = df_ipo_2.employees.astype('float64')

#Mean imputing offer price, sector, CEO age and employee columns
df_ipo_2['CEO_age'].fillna(df_ipo_2['CEO_age'].mean(), inplace=True)
df_ipo_2['Price'].fillna(df_ipo_2['Price'].mean(), inplace=True)
df_ipo_2['employees'].fillna(df_ipo_2['employees'].mean(), inplace=True)

#Adding Other sector for missing sectors
df_ipo_2['sector'].fillna('Other', inplace = True)

#Dropping all remaining cols with missing data
df_ipo_2.dropna(inplace=True)
df_ipo_2.isna().sum()

Symbol             0
IPO_weekday        0
closeDay0          0
Price              0
closeDay6          0
closeDay29         0
Name               0
sector             0
ipoDate            0
CEO_age            0
employees          0
firstday_volume    0
inweek_volume      0
inmonth_volume     0
intra_day_up       0
intra_week_up      0
intra_month_up     0
dtype: int64

In [372]:
from datetime import timedelta

#Adding computed trading volumes
vol_check = df_ipo_2[['Symbol','firstday_volume', 'inweek_volume', 'inmonth_volume']]

df_ipo = df_ipo.merge(vol_check, how = 'left', on = 'Symbol')

#Computing missing trading volumes using yfinance

#defining function for volume extractions
def getVolumes(Symbol, Date):
    print(Symbol)
    ipo_date = pd.to_datetime(Date)
    try:
        data = yf.download(Symbol, ipo_date, ipo_date + timedelta(days= 29))['Volume']
    except:
        return (np.NAN,np.NAN,np.NAN)
    if data.empty:
        return (np.NAN,np.NAN,np.NAN)
    first_day_volume = data[0]
    inweek_volume = sum(data[:7])
    inmonth_volume = sum(data[:30])
    return (first_day_volume, inweek_volume, inmonth_volume)

df_missing_vol = df_ipo.loc[df_ipo.firstday_volume.isna()].copy()
df_missing_vol[['firstday_volume', 'inweek_volume', 'inmonth_volume']] = df_missing_vol.apply(lambda x: getVolumes(x['Symbol'], x['Date Priced']), axis = 1, result_type='expand')

AMCF
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AMCF: No data found for this date range, symbol may be delisted
CRMD
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- CRMD: No data found for this date range, symbol may be delisted
ELMD
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- ELMD: No data found for this date range, symbol may be delisted
STND
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- STND: No timezone found, symbol may be delisted
GNOM
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- GNOM: No data found for this date range, symbol may be delisted
TBET
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TBET: No data found for this date range, symbol may be delisted
BGMD
[*********************100%***********************] 

In [373]:
#Joining in additional trading volumes
df_ipo.fillna(df_missing_vol, inplace=True)

#Filling in remaining NaNs with mean volumes
df_ipo['firstday_volume'].fillna(df_ipo['firstday_volume'].mean(), inplace=True)
df_ipo['inweek_volume'].fillna(df_ipo['inweek_volume'].mean(), inplace=True)
df_ipo['inmonth_volume'].fillna(df_ipo['inmonth_volume'].mean(), inplace=True)



In [374]:
#Writing clean file to disk
df_ipo.to_csv(path + 'clean_' + data_2018)

In [375]:
#Creating a larger dataset with lower amount of features from df_ipo and df_ipo_2

#Dropping unneeded columns in both datasets
df_ipo_2.drop(columns=['closeDay0', 'closeDay6', 'closeDay29', 'Name'], inplace= True)
df_ipo.drop(columns= ['CEO_pay', 'Market', 'Shares'], inplace= True)

#Renaming columns so dfs match
df_ipo.rename(columns={'Date Priced': 'ipoDate'}, inplace=True)

#Concatenating dataframes
df_master = pd.concat([df_ipo, df_ipo_2.loc[~df_ipo_2.Symbol.isin(df_ipo.Symbol)]], axis = 0)
df_master

,Price,employees,CEO_age,intra_day_up,intra_week_up,intra_month_up,Symbol,ipoDate,sector,IPO_weekday,firstday_volume,inweek_volume,inmonth_volume
0,6.3000,128.000000,57.460211,1,1,0,AMCF,2010-01-26,Energy,1,9.974033e+06,6.223814e+06,1.703233e+07
1,20.0000,3.000000,60.000000,1,1,1,CHSP,2010-01-22,Real Estate,4,3.492458e+06,6.004700e+05,1.911825e+06
2,13.0000,1486.000000,50.000000,1,1,1,GNRC,2010-02-11,Industrials,3,1.689209e+07,2.179070e+06,4.953062e+06
3,15.0000,568.000000,62.000000,0,1,0,QNST,2010-02-11,Technology,3,5.372077e+06,3.361885e+06,7.002853e+06
4,20.0000,6.000000,61.000000,1,1,1,TRNO,2010-02-10,Real Estate,2,4.515261e+06,7.613210e+05,1.937712e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3749,7.0500,28.000000,50.000000,0,0,0,ZN,1/3/07 0:00,Energy,2,2.630000e+04,2.028000e+05,6.576580e+05
3751,20.6667,7293.424463,54.812500,0,0,0,ZNH,7/30/97 0:00,Transportation,2,4.703100e+06,2.526750e+06,7.261650e+06
3752,23.7300,5473.000000,49.000000,0,0,0,ZOES,4/11/14 0:00,Consumer Services,4,5.765164e+06,2.644271e+06,6.955747e+06
3757,10.2500,8900.000000,57.000000,0,0,0,ZUMZ,5/6/05 0:00,Consumer Services,4,6.793424e+06,1.954885e+06,7.014295e+06
